# Find Clips to process

In [13]:
import json

def is_hand(obj):
    hand_objects = {'left hand', 'right hand', 'both hands', 'left_hand', 'right_hand', 'both_hands'}
    return obj in hand_objects

def analyze_clips(json_data):
    scored_clips = []

    for clip_id, clip_info in json_data.items():
        graphs = clip_info.get('graphs', [])
        if not graphs:
            continue
        
        unique_objects = set()
        sequence_length = len(graphs)
        
        for graph in graphs:
            for triplet in graph.get('triplets', []):
                if len(triplet) < 3:
                    continue
                obj = triplet[2]
                if not is_hand(obj):
                    unique_objects.add(obj)
        
        unique_count = len(unique_objects)
        if unique_count == 0:
            continue
        
        score = sequence_length * unique_count
        scored_clips.append({
            'clip_id': clip_id,
            'score': score,
            'sequence_length': sequence_length,
            'unique_objects': unique_count,
            'objects': list(unique_objects)
        })
    
    scored_clips.sort(key=lambda x: (-x['score'], -x['sequence_length'], -x['unique_objects']))
    return scored_clips[:5]

def find_most_interesting(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    top_clips = analyze_clips(data)
    
    print("Top 5 most interesting clips:")
    for i, clip in enumerate(top_clips, 1):
        print(f"{i}. Clip ID: {clip['clip_id']}")
        print(f"   Score: {clip['score']} (Sequence length: {clip['sequence_length']}, Unique objects: {clip['unique_objects']})")
        print(f"   Objects involved: {', '.join(clip['objects'])}\n")


find_most_interesting('EASG_unict_master_final.json')

Top 5 most interesting clips:
1. Clip ID: c1ef9d03-1278-4216-b1e4-b1a298a892d9
   Score: 5805 (Sequence length: 215, Unique objects: 27)
   Objects involved: carry, pack, roll up, separate, rack, seperate, lid, cut, tray, cutter, move, pick, flour, scraper, dough strip, drop, knead, detach, remove, dough strips, table, mixture, join, dough, close, pass, mixure

2. Clip ID: e8e54409-495b-49e5-b325-556c812d6ff4
   Score: 4788 (Sequence length: 133, Unique objects: 36)
   Objects involved: tap, pick up, bowl, pour off, turn off, clean, apply, plates, steel, tray, move, zinc, scrubber, soap, raise, water, knife, drop up, sponge, drop, lower, rinse, open, sink faucet, scourer, fork, keep, sink, plate, slab, wash, hold, plate:1, dish, spoon, turn on

3. Clip ID: a4385c9b-84f0-4d8f-9227-1efe9f264d50
   Score: 4692 (Sequence length: 102, Unique objects: 46)
   Objects involved: tap, take out, carry, pick up, fetch, place, paper, empty, put out, cut, put in, countertop, stick, tray, move, pick,

In [14]:
import json

def is_hand(obj):
    hand_objects = {'left hand', 'right hand', 'both hands'}
    return obj in hand_objects

def analyze_clips(json_data, count_lengths=True, topk=5):
    scored_clips = []
    
    for clip_id, clip_info in json_data.items():
        graphs = clip_info.get('graphs', [])
        if not graphs:
            continue
        
        sequence_length = len(graphs)
        unique_verbs = set()
        unique_objects = set()
        indirect_object_count = 0
        
        for graph in graphs:
            for triplet in graph.get('triplets', []):
                if len(triplet) < 3:
                    continue
                
                # Handle verbs from both action triplets and CW-verb triplets
                if triplet[0] == 'CW' and triplet[1] == 'verb':
                    unique_verbs.add(triplet[2])
                else:
                    verb = triplet[0]
                    relation = triplet[1]
                    obj = triplet[2]
                    unique_verbs.add(verb)
                    
                    # Track objects and indirect relations
                    if not is_hand(obj):
                        unique_objects.add(obj)
                        if relation != 'dobj':
                            indirect_object_count += 1
        
        unique_verbs_count = len(unique_verbs)
        unique_objects_count = len(unique_objects)
        
        if unique_objects_count == 0 and indirect_object_count == 0:
            continue  # Skip clips without relevant objects
        
        # Calculate scores based on the formula
        base_score = unique_verbs_count + unique_objects_count + indirect_object_count
        score = base_score * sequence_length if count_lengths else base_score
        
        scored_clips.append({
            'clip_id': clip_id,
            'score': score,
            'sequence_length': sequence_length,
            'unique_verbs': unique_verbs_count,
            'unique_objects': unique_objects_count,
            'indirect_objects': indirect_object_count,
            'verbs': list(unique_verbs),
            'objects': list(unique_objects)
        })
    
    # Sort by score descending, then sequence length, then object diversity
    scored_clips.sort(key=lambda x: (-x['score'], -x['sequence_length'], -x['unique_objects']))
    return scored_clips[:topk]

def find_most_interesting_v2(json_file, count_lengths=True, topk = 5):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    top_clips = analyze_clips(data, count_lengths, topk=topk)
    
    print(f"Top 5 most interesting clips (count_lengths={count_lengths}):")
    for i, clip in enumerate(top_clips, 1):
        print(f"{i}. Clip ID: {clip['clip_id']}")
        print(f"   Score: {clip['score']} (Sequence: {clip['sequence_length']}, Verbs: {clip['unique_verbs']}, Objects: {clip['unique_objects']}, Indirect: {clip['indirect_objects']})")
        print(f"   Verbs: {', '.join(clip['verbs'])}")
        print(f"   Objects: {', '.join(clip['objects'])}\n")
    
    return top_clips

most_interesting_clips_2 = find_most_interesting_v2('EASG_unict_master_final.json', False, 1000)

Top 5 most interesting clips (count_lengths=False):
1. Clip ID: 49d65bed-9479-427e-bb68-0c9a18560ac6
   Score: 110 (Sequence: 157, Verbs: 13, Objects: 8, Indirect: 89)
   Verbs: arrange, cut, drop, pick up, move, pick, place, adjust, cardboard, put away, press, clean off, lift
   Objects: brush, card, side, cutter, cup, cardboard, ruler, table

2. Clip ID: 2ae97f2a-b16d-467b-b73a-721cb00996f1
   Score: 100 (Sequence: 94, Verbs: 22, Objects: 24, Indirect: 54)
   Verbs: carry, pick up, bowl, apply, place, pour out, mix, scoop, cut, bring out, roll, touch, move, pick, adjust, rub, drop, take, knead, open, put, pour
   Objects: brush, bowl, bag, scoop, lid, mixer, countertop, cutter, tray, pieces, cup, bucket, pan, scale, flour, scraper, dough strip, floor, bottle, table, trey, oil, dough, scrapper

3. Clip ID: fc162164-d4c6-4fdb-a511-f3a500ec031e
   Score: 94 (Sequence: 117, Verbs: 15, Objects: 21, Indirect: 58)
   Verbs: add, pick up, cut, put in, pick sup, touch, pull, move, pick, place

In [3]:
most_interesting_clips_2 = find_most_interesting_v2('EASG_unict_master_final.json', False, 20)

Top 5 most interesting clips (count_lengths=False):
1. Clip ID: 49d65bed-9479-427e-bb68-0c9a18560ac6
   Score: 110 (Sequence: 157, Verbs: 13, Objects: 8, Indirect: 89)
   Verbs: arrange, cut, drop, pick up, move, pick, place, adjust, cardboard, put away, press, clean off, lift
   Objects: brush, card, side, cutter, cup, cardboard, ruler, table

2. Clip ID: 2ae97f2a-b16d-467b-b73a-721cb00996f1
   Score: 100 (Sequence: 94, Verbs: 22, Objects: 24, Indirect: 54)
   Verbs: carry, pick up, bowl, apply, place, pour out, mix, scoop, cut, bring out, roll, touch, move, pick, adjust, rub, drop, take, knead, open, put, pour
   Objects: brush, bowl, bag, scoop, lid, mixer, countertop, cutter, tray, pieces, cup, bucket, pan, scale, flour, scraper, dough strip, floor, bottle, table, trey, oil, dough, scrapper

3. Clip ID: fc162164-d4c6-4fdb-a511-f3a500ec031e
   Score: 94 (Sequence: 117, Verbs: 15, Objects: 21, Indirect: 58)
   Verbs: add, pick up, cut, put in, pick sup, touch, pull, move, pick, place

# Select unique actions

In [15]:
import json

def process_annotations(input_file, selected_clips=None):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    output = {}

    if selected_clips is None:
        selected_clips = data.keys()
    
    for clip_uid in selected_clips:
        clip_data = data[clip_uid]
        actions = {}
        graphs = clip_data.get("graphs", [])
        for idx, graph in enumerate(graphs, start=1):
            verb = None
            obj = None
            indirects = []
            
            for triplet in graph.get("triplets", []):
                if triplet[0] == "CW" and triplet[1] == "verb":
                    verb = triplet[2]
                elif triplet[1] == "dobj":
                    obj = triplet[2]
                else:
                    relation = triplet[1]
                    value = triplet[2]
                    if "hand" in value.lower():
                        continue
                    indirects.append(f"{relation} {value}")
            
            if verb is not None and obj is not None:
                action_entry = {"verb": verb, "object": obj}
                if indirects:
                    action_entry["indirect"] = ", ".join(indirects)
                actions[idx] = action_entry
        
        output[clip_uid] = actions
    
    return output

# Example usage:
selected_clips = [clip['clip_id'] for clip in most_interesting_clips_2]
processed_data = process_annotations('EASG_unict_master_final.json', selected_clips)
print(json.dumps(processed_data, indent=2))

{
  "49d65bed-9479-427e-bb68-0c9a18560ac6": {
    "1": {
      "verb": "place",
      "object": "cardboard",
      "indirect": "on table"
    },
    "2": {
      "verb": "place",
      "object": "cardboard",
      "indirect": "on table"
    },
    "3": {
      "verb": "place",
      "object": "cardboard",
      "indirect": "on table"
    },
    "4": {
      "verb": "place",
      "object": "cardboard",
      "indirect": "on table"
    },
    "5": {
      "verb": "place",
      "object": "cardboard",
      "indirect": "on table"
    },
    "6": {
      "verb": "place",
      "object": "cardboard"
    },
    "7": {
      "verb": "press",
      "object": "cardboard"
    },
    "8": {
      "verb": "drop",
      "object": "cardboard"
    },
    "9": {
      "verb": "pick",
      "object": "cup"
    },
    "10": {
      "verb": "pick",
      "object": "cup"
    },
    "11": {
      "verb": "pick",
      "object": "cup"
    },
    "12": {
      "verb": "pick",
      "object": "cardboard",
  

In [16]:
from collections import Counter
def mark_repeated_values(input_dict):
    # Convert dict values to JSON strings for counting
    serialized_values = {k: json.dumps(v, sort_keys=True) for k, v in input_dict.items()}
    
    # Count occurrences of each serialized value
    value_counts = Counter(serialized_values.values())
    
    # Assign "unique" or "repeat" based on occurrences
    return {key: "unique" if value_counts[serialized_values[key]] == 1 else "repeat" for key in input_dict}

marked_data = {}
total_unique = 0
filtered_data = {}
for clip_uid, data in processed_data.items():
    marked_data[clip_uid] = mark_repeated_values(data)
    filtered_data[clip_uid] = {}
    for k, v in marked_data[clip_uid].items():
        if v == 'unique':
            filtered_data[clip_uid][k] = processed_data[clip_uid][k]
    num_unique = sum(1 for v in marked_data[clip_uid].values() if v == 'unique')
    total = len(marked_data[clip_uid])
    total_unique += num_unique
    print(f"Clip {clip_uid}: {num_unique} unique actions out of {total}")
print(f"Total unique actions: {total_unique}")

Clip 49d65bed-9479-427e-bb68-0c9a18560ac6: 16 unique actions out of 157
Clip 2ae97f2a-b16d-467b-b73a-721cb00996f1: 36 unique actions out of 94
Clip fc162164-d4c6-4fdb-a511-f3a500ec031e: 28 unique actions out of 117
Clip 4f68183f-610a-44de-b102-e7f300b49dcd: 50 unique actions out of 87
Clip e43f5b38-213a-4d42-afaa-32c3b77f4da6: 43 unique actions out of 74
Clip c1ef9d03-1278-4216-b1e4-b1a298a892d9: 28 unique actions out of 214
Clip 8cf5a53b-fa9c-4fe7-b767-697133d4d5b8: 11 unique actions out of 85
Clip a4385c9b-84f0-4d8f-9227-1efe9f264d50: 35 unique actions out of 102
Clip e8cf9894-0cc7-4480-b490-885308bfb513: 25 unique actions out of 77
Clip 4ad40842-5cee-49c2-a382-211fd7a4872b: 26 unique actions out of 47
Clip e8e54409-495b-49e5-b325-556c812d6ff4: 27 unique actions out of 133
Clip 203ee455-6f60-4e08-bd89-28e2ca3927da: 33 unique actions out of 85
Clip 3c18ec5c-7e6b-400d-9260-70598e04a373: 28 unique actions out of 73
Clip 6f082d5d-5f31-4358-b2dc-16d320312ab3: 32 unique actions out of 105


In [ ]:
print(json.dumps(filtered_data, indent=2))

In [17]:
pairs_finding_prompt = """
**Task**: Analyze a set of actions and label them as 'repeat', 'check', or 'unique' based on the repetitiveness of verb-object relationships and indirect relations.

**Definitions**:
1. `repeat`: Same/synonymous verb + object pair with identical/no indirect relations  
   (e.g., {"verb":"place", "object":"wood"} vs {"verb":"put", "object": "wooden piece")
2. `check`: Same/synonymous verb + object pair BUT different indirect relations  
   (e.g., "put wood (on table)" vs "place wood (on book)")
3. `unique`: No matching verb-object pairs in other actions from the set

**Instructions**:
1. Compare ALL action pairs across the entire dataset
2. Consider indirect relations ONLY for 'check' determination
3. Return ONLY JSON in the format: {"action_number": "label"}. NO OTHER TEXT NEEDED.

**Example**
Input:
{
  "1": {"verb": "place", "object": "wood"},
  "2": {"verb": "put", "object": "wooden piece", "indirect": "on table"},
  "3": {"verb": "grab", "object": "book", "indirect": "from table"},
  "4": {"verb": "pick", "object": "drill"},
  "24": {"verb": "take", "object": "notebook", "indirect": "from shelf"}
}

Output:
{
  "1": "repeat",
  "2": "repeat",
  "3": "check",
  "4": "unique",
  "24": "check"
}
**End of Example**

Please complete the task below and provide the output JSON.

Input:

"""

In [18]:
import json
import re
def extract_json_from_string(input_string):
    json_match = re.search(r"\{.*\}", input_string, re.DOTALL)
    if json_match:
        # json_str = json_match.group(0)  # Extract matched JSON part
        json_matches = re.findall(r"\{.*?\}", input_string, re.DOTALL)
        if not json_matches:
            return None  # No JSON found
        json_str = max(json_matches, key=len)
        json_str = json_str.replace("'", '"')  # Fix single quotes
        json_str = json_str.replace("\n", "")  # Fix single quotes
        json_str = re.sub(r"(\{|,)\s*(\d+)\s*:", r'\1 "\2":', json_str)

        # print(json_str)
        try:
            return json.loads(json_str)  # Parse to dict
        except json.JSONDecodeError:
            print("Error: Extracted text is not valid JSON.")
            return None
    return None

def combine_5_outputs(llm_outputs):
    """
    function takes a list of 5 outputs from the LLM, extracts jsons from them and compares them
    for each key in the json it assigns values 'unique':0, 'check':1, 'repeat':2
    then it sums the values for each key and returns the dictionary with the sums
    """
    outputs = [extract_json_from_string(output) for output in llm_outputs]
     # Identify how many failed
    # Print a warning if some outputs are None
    if outputs.count(None) > 0:
        print(f"Warning: {outputs.count(None)} out of 5 outputs could not be parsed.")

    # Filter out None values
    outputs = [output for output in outputs if output is not None]

    N = len(outputs)
    
    combined_data = {}
    for output in outputs:
        for key, value in output.items():
            if key not in combined_data:
                combined_data[key] = []
            combined_data[key].append(value)
    
    result = {}
    label2score = {"unique":0, "check":1, "repeat":2}
    for key, values in combined_data.items():
        try:
            result[key] = sum([label2score[value] for value in values])/(2*N)
        except KeyError:
            result[key] = 100  # Assign a high value for unknown labels
    return result

In [19]:
import requests
import time
import re

# def trim_string(s):
#     match = re.search(r'\{.*?\}', s)
#     return match.group(0) if match else ''

# Define the API endpoint and model
api_url = "http://192.168.90.11:5343/api/generate"
model_name = "llama3.1:8b-instruct-fp16" 
# model_name = "llama3:8b"

selected_fields = ['verb', 'relation', 'obj', 'additional', 'answer']

action_score_outputs = {}

times = []
for clip_uid, data in filtered_data.items():
    llm_outputs = []
    start_time = time.time()
    payload = {
        "model": model_name,
        "prompt": pairs_finding_prompt + str(data),
        "temperature": 0.7,
        "stream": False  # Set to True if you want to stream the response
    }

    for _ in range(10):
        # Send the POST request to the Ollama server
        response = requests.post(api_url, json=payload)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()
            
            # Extract and print the generated text
            generated_text = response_data.get("response", "")
            llm_outputs.append(generated_text)
            # print("Generated Text:", generated_text)
        else:
            print(f"Failed to get a response. Status code: {response.status_code}")
            print("Response:", response.text)
    end_time = time.time()
    times.append(end_time-start_time)
    print(f"Clip {clip_uid} processed in {end_time-start_time:.2f} seconds")
    action_score_outputs[clip_uid] = combine_5_outputs(llm_outputs)
    print("Output:", action_score_outputs[clip_uid])
    # print(llm_outputs)

# Save the outputs to a JSON file
with open("action_score_outputs.json", "w") as f:
    json.dump(action_score_outputs, f, indent=2)

Clip 49d65bed-9479-427e-bb68-0c9a18560ac6 processed in 42.58 seconds
Output: {'7': 1.0, '8': 0.7, '23': 0.0, '25': 0.5, '33': 0.4, '38': 0.55, '45': 0.85, '52': 0.6, '64': 0.9, '81': 0.75, '84': 0.1, '95': 0.5, '97': 0.5, '107': 0.5, '109': 0.3, '139': 0.55}
Clip 2ae97f2a-b16d-467b-b73a-721cb00996f1 processed in 68.77 seconds
Output: {'1': 0.9, '4': 0.0, '9': 0.6, '13': 0.5, '15': 0.6, '22': 0.55, '26': 0.4, '35': 0.0, '39': 0.5, '40': 0.7, '42': 0.55, '44': 0.45, '52': 0.1, '55': 0.5, '56': 0.1, '57': 0.5, '60': 0.5, '61': 0.35, '62': 0.5, '63': 0.7, '64': 0.4, '68': 0.0, '69': 0.0, '77': 0.15, '79': 0.0, '81': 0.35, '82': 0.8, '83': 0.5, '84': 0.5, '86': 0.5, '87': 0.65, '88': 0.5, '91': 1.0, '92': 1.0, '93': 0.0, '94': 0.5}
Clip fc162164-d4c6-4fdb-a511-f3a500ec031e processed in 52.32 seconds
Output: {'7': 0.4, '17': 0.9, '18': 0.65, '21': 0.3, '25': 0.8, '31': 0.55, '32': 0.4, '33': 0.55, '37': 0.2, '43': 0.55, '44': 0.7, '46': 0.7, '49': 0.3, '50': 0.55, '57': 0.1, '58': 0.0, '61':

In [ ]:
x = ['To analyze the actions in the video clip and label them as \'repeat\', \'check\', or \'unique\' based on verb-object relationships and indirect relations, I\'ll follow the instructions and definitions provided.\n\nHere\'s the output:\n```\n{\n  7: "unique",\n  8: "repeat",\n  23: "unique",\n  25: "repeat",\n  33: "unique",\n  38: "check",\n  45: "unique",\n  52: "unique",\n  64: "unique",\n  81: "unique",\n  84: "unique",\n  95: "check",\n  97: "check",\n  107: "unique",\n  109: "unique"\n}\n```\nHere\'s the analysis for each action:\n\n1. **7**: press cardboard (unique)\n2. **8**: drop cardboard (repeat, same verb-object pair as action 7)\n3. **23**: place ruler (unique)\n4. **25**: adjust ruler (repeat, same verb-object pair as action 23)\n5. **33**: pick card (unique)\n6. **38**: pick up cardboard (indirect relation: from side, with cutter) -> check\n7. **45**: pick up cardboard (unique)\n8. **52**: cut cardboard (indirect relation: from side, with cutter) -> unique\n9. **64**: arrange cardboard (unique)\n10. **81**: drop cutter (unique)\n11. **84**: clean off cardboard (unique)\n12. **95**: cut cardboard (indirect relation: with cutter, on table) -> check\n13. **97**: cut cardboard (indirect relation: on table, with ruler) -> check\n14. **107**: cut cardboard (indirect relation: in cutter) -> unique\n15. **109**: put away cardboard (unique)\n\nNote that I\'ve considered indirect relations only for \'check\' determination and used the synonym mappings provided to identify matching verb-object pairs.', 'I\'ll analyze the actions in the video clip and label them as \'repeat\', \'check\', or \'unique\' based on verb-object relationships and indirect relations.\n\nHere is the output in JSON format:\n```\n{\n 7: "unique",\n 8: "repeat",\n 23: "unique",\n 25: "repeat",\n 33: "unique",\n 38: "check",\n 45: "unique",\n 52: "unique",\n 64: "unique",\n 81: "unique",\n 84: "unique",\n 95: "check",\n 97: "check",\n 107: "unique",\n 109: "unique"\n}\n```\nHere\'s the breakdown:\n\n* Action 7, 45, and 107 have unique verb-object pairs with no matching indirect relations.\n* Actions 8 and 25 have repeat verb-object pairs (drop cardboard and place ruler) but differ in indirect relations (none vs. on table).\n* Actions 38, 95, 97 have check labels because they share the same verb-object pair (cut cardboard) but differ in indirect relations (on side with cutter, on table with cutter, and on table with ruler respectively).\n* All other actions have unique verb-object pairs or no matching indirect relations.', 'I\'ll analyze the actions and label them as \'repeat\', \'check\', or \'unique\' based on verb-object relationships and indirect relations.\n\nHere\'s the output in JSON format:\n\n{\n"7": "unique",\n"8": "repeat",\n"23": "unique",\n"25": "check",\n"33": "unique",\n"38": "unique",\n"45": "unique",\n"52": "check",\n"64": "unique",\n"81": "unique",\n"84": "unique",\n"95": "check",\n"97": "unique",\n"107": "unique",\n"109": "unique"\n}\n\nHere\'s the explanation for each action:\n\n* 7: Unique - No matching verb-object pair.\n* 8: Repeat - Same verb ("drop") and object ("cardboard") as in action 45.\n* 23: Unique - No matching verb-object pair.\n* 25: Check - Same verb ("adjust") but different indirect relation ("on ruler") compared to action 97.\n* 33: Unique - No matching verb-object pair.\n* 38: Unique - No matching verb-object pair.\n* 45: Unique - No matching verb-object pair.\n* 52: Check - Same verb ("cut") and object ("cardboard") as in actions 95 and 97, but different indirect relations.\n* 64: Unique - No matching verb-object pair.\n* 81: Unique - No matching verb-object pair.\n* 84: Unique - No matching verb-object pair.\n* 95: Check - Same verb ("cut") and object ("cardboard") as in action 52, but different indirect relation.\n* 97: Unique - No matching verb-object pair.\n* 107: Unique - No matching verb-object pair.\n* 109: Unique - No matching verb-object pair.\n\nLet me know if you have any questions or need further clarification!', 'I\'ll analyze the actions in the video clip and label them as \'repeat\', \'check\', or \'unique\' based on verb-object relationships and indirect relations. Here\'s my output:\n\n```\n{\n 7: "unique",\n 8: "repeat", \n 23: "unique",\n 25: "check", \n 33: "unique",\n 38: "check", \n 45: "unique", \n 52: "check", \n 64: "unique", \n 81: "unique", \n 84: "unique", \n 95: "check", \n 97: "unique", \n 107: "unique", \n 109: "unique",\n 139: "unique"\n}\n```\n\nHere\'s my reasoning for each action:\n\n1. Action 7: The verb-object pair ("press" and "cardboard") doesn\'t match any other actions, so it\'s labeled as \'unique\'.\n2. Action 8: The verb-object pair ("drop" and "cardboard") is identical to Action 45, so it\'s labeled as \'repeat\'.\n3. Action 23: The verb-object pair ("place" and "ruler") doesn\'t match any other actions, so it\'s labeled as \'unique\'.\n4. Action 25: The verb-object pair ("adjust" and "ruler") is different from the indirect relation in Actions 97 and 107, but similar to the verb-object pair in Action 23. Therefore, it\'s labeled as \'check\'.\n5. Action 33: The verb-object pair ("pick" and "card") doesn\'t match any other actions, so it\'s labeled as \'unique\'.\n6. Action 38: The verb-object pair ("pick up" and "cardboard") is different from the indirect relation in Actions 52 and 95, but similar to the verb-object pair in Action 45. Therefore, it\'s labeled as \'check\'.\n7. Action 45: The verb-object pair ("drop" and "cardboard") matches Action 8, so it\'s labeled as \'repeat\'.\n8. Action 52: The verb-object pair ("cut" and "cardboard", with indirect relation) is different from the other actions, so it\'s labeled as \'unique\'.\n9. Action 64: The verb-object pair ("arrange" and "cardboard") doesn\'t match any other actions, so it\'s labeled as \'unique\'.\n10. Action 81: The verb-object pair ("drop" and "cutter") doesn\'t match any other actions, so it\'s labeled as \'unique\'.\n11. Action 84: The verb-object pair ("clean off" and "cardboard") doesn\'t match any other actions, so it\'s labeled as \'unique\'.\n12. Action 95: The verb-object pair ("cut" and "cardboard", with indirect relation) is different from the other actions, but similar to Actions 52 and 107. Therefore, it\'s labeled as \'check\'.\n13. Action 97: The verb-object pair ("cut" and "cardboard", with indirect relation) is different from the other actions, so it\'s labeled as \'unique\'.\n14. Action 107: The verb-object pair ("cut" and "cardboard", with indirect relation) matches Actions 95 and 52, but has a different indirect relation. Therefore, it\'s labeled as \'check\'.\n15. Action 109: The verb-object pair ("put away" and "cardboard") doesn\'t match any other actions, so it\'s labeled as \'unique\'.\n16. Action 139: The verb-object pair ("lift" and "ruler", with indirect relation) is different from the other actions, so it\'s labeled as \'unique\'.\n\nLet me know if you\'d like me to clarify or justify any of these labels!', 'After analyzing the actions in the video clip and applying the definitions and instructions provided, I\'ve generated the following output:\n\n{\n7: "unique",\n8: "repeat",\n23: "unique",\n25: "check",\n33: "unique",\n38: "check",\n45: "repeat",\n52: "check",\n64: "unique",\n81: "repeat",\n84: "unique",\n95: "check",\n97: "check",\n107: "unique",\n109: "unique",\n139: "repeat"\n}\n\nHere\'s a breakdown of each action:\n\n* Action 7: "press cardboard" - unique (no matching verb-object pair)\n* Action 8: "drop cardboard" - repeat (same verb-object pair as action 1)\n* Action 23: "place ruler" - unique (no matching verb-object pair)\n* Action 25: "adjust ruler" - check (different indirect relation from action 23)\n* Action 33: "pick card" - unique (no matching verb-object pair)\n* Action 38: "pick up cardboard with cutter" - check (different indirect relation from actions involving cardboard or cutter alone)\n* Action 45: "pick up cardboard" - repeat (same verb-object pair as action 8)\n* Action 52: "cut cardboard with cutter, from side" - check (different indirect relation from actions involving cutting or ruler)\n* Action 64: "arrange cardboard" - unique (no matching verb-object pair)\n* Action 81: "drop cutter" - repeat (same verb-object pair as action 8)\n* Action 84: "clean off cardboard" - unique (no matching verb-object pair)\n* Action 95: "cut cardboard with ruler, on table" - check (different indirect relation from actions involving cutting or ruler)\n* Action 97: "cut cardboard with ruler, on table" - check (same indirect relation as action 95)\n* Action 107: "cut cardboard in cutter" - unique (no matching verb-object pair)\n* Action 109: "put away cardboard" - repeat (same verb-object pair as action 45)\n* Action 139: "lift ruler off cardboard, from table" - check (different indirect relation from actions involving lifting or ruler)\n\nNote that I\'ve considered only the provided synonym mappings and definitions for labeling actions. If you\'d like to add more synonyms or refine the definitions, please let me know!']

for text in x:
    print(extract_json_from_string(text))

In [20]:
PROMPT_DIRECT_QUESTION = """
Your task is to generate a question based on json containing verb, indirect relations.
Basically, you need to compose a question from "verb" and "indirect".
The questions actor on the question should be Camera Wearer (CW).

Note: if the objects have numbers (e.g. "book:0", "book:1"), you have to keep them in question as is (e.g. "What does CW pick from book:0?")

IMPORTANT: Output just the question, without additional text.
Examples:

Input: {'verb': 'rub', 'indirect': 'with brush, on dough'}
Output: "What does CW rub with a brush on dough?"

Input: {'verb': 'place', 'indirect': 'on table'}
Output: "What does CW place on the table?"

Input: {'verb': 'pick', 'indirect': 'from deck:0'}
Output: "What does CW pick from the deck:0?"

Input: {'verb': 'cut', 'indirect': 'with scissors:1'}
Output: "What does CW cut with scissors:1?"
"""

PROMPT_INDIRECT_QUESTION = """
Your task is to generate a question based on json containing verb, object and indirect relations.
You know the answer in advance, it is stored in the 'answer' field. But you should not include it in the question!

Basically, you need to compose a question from "verb", "object", and "other_relations", in a way that this question can be answered with the "answer".
Note: if the objects have numbers (e.g. "book:0", "book:1"), you have to keep them in question as is (e.g. "Where does CW pick the card from book:0?")

IMPORTANT: Output must contain ONLY the question, without additional text.

Examples:

Input: {'verb': 'rub', 'object': 'oil', 'answer': 'with brush', 'other_relations': 'on dough'}
Output: "With what does CW rub the oil on dough?"

Input: {'verb': 'place', 'object': 'wood', 'answer': 'on countertop'}
Output: "Where does CW place the wood?"

Input: {'verb': 'pick', 'object': 'card', 'answer': 'from shelf'}
Output: "Where does CW pick the card from?"

Input: {'verb': 'cut', 'object': 'cardboard', 'other_relations': 'on table', 'answer': 'with cutter'}
Output: "What does CW use to cut the cardboard on the table?"
"""

PROMPT_BEFORE = """
Your task is to generate a question based on json containing verb, object and indirect relations.
Basically, you need to compose a question from "verb", "object", and "indirect".
The questions actor on the question should be Camera Wearer (CW).
The question should be in the format: "What did camera wearer do before the action?"

Note: if the objects have numbers (e.g. "book:0", "book:1"), you have to keep them in question as is (e.g. "What did CW pick from book:0 before the action?")
IMPORTANT: Output just the question, without additional text.

Examples:

Input: {'verb': 'rub', 'object': 'oil', 'indirect': 'with brush', 'other_relations': 'on dough'}
Output: "What did camera wearer do before rubbing the oil with a brush on dough?"

Input: {'verb': 'place', 'object': 'wood', 'indirect': 'on table'}
Output: "What did camera wearer do before placing the wood on the table?"

Input: {'verb': 'pick', 'object': 'card', 'indirect': 'from deck:0'}
Output: "What did camera wearer do before picking the card from the deck:0?"
"""

PROMPT_AFTER = """
Your task is to generate a question based on json containing verb, object and indirect relations.
Basically, you need to compose a question from "verb", "object", and "indirect".
The questions actor on the question should be Camera Wearer (CW).
The question should be in the format: "What did camera wearer do after the action?"

Note: if the objects have numbers (e.g. "book:0", "book:1"), you have to keep them in question as is (e.g. "What did CW pick from book:0 after the action?")

IMPORTANT: Output just the question, without additional text.

Examples:

Input: {'verb': 'rub', 'object': 'oil', 'indirect': 'with brush', 'other_relations': 'on dough'}
Output: "What did camera wearer do after rubbing the oil with a brush on dough?"

Input: {'verb': 'place', 'object': 'wood', 'indirect': 'on table'}
Output: "What did camera wearer do after placing the wood on the table?"

Input: {'verb': 'pick', 'object': 'card', 'indirect': 'from deck:0'}
Output: "What did camera wearer do after picking the card from the deck:0?"
"""

PROMPT_PURPOSE = """
Your task is to generate question-answer pairs for the purpose of objects, mentioned in the sequence of actions.
Actions are annotations of the video clip, recorded by the camera wearer (CW).
Each action consists of a verb, object, and indirect relation.

Select the most interesting objects/actions from the clip and generate questions about their purpose.
It's better to generate less, but more interesting questions. Two - six Q-A pairs are enough.

The outputs should be in the format of valid JSON in the format as below:

{"drill":
    {"question": "What is the purpose of the drill on the video?",
    "answer": "The purpose of the drill is to make holes in the chair."}
"screwdriver":
    {"question": "What is the purpose of the screwdriver in the video?",
    "answer": "The screwdriver is used to tighten the screws in the table."}
}

IMPORTANT: All the answers should be anwerable from the input data! Don't made up answers!

IMPORTANT: Output just the valid JSON, without additional text!
"""


In [21]:
from tqdm import tqdm

# read json action_score_outputs
with open("action_score_outputs.json", "r") as f:
    action_scores = json.load(f)

import requests
# Define the API endpoint and model
api_url = "http://192.168.90.11:5343/api/generate"
# model_name = "llama3.1:8b-instruct-fp16" 
model_name = "llama3:8b"

def generate_question(data, prompt):
    payload = {
        "model": model_name,
        "prompt": prompt + str(data),
        "temperature": 0.7,
        "stream": False  # Set to True if you want to stream the response
    }
    response = requests.post(api_url, json=payload)
    if response.status_code == 200:
        # Parse the JSON response
        response_data = response.json()
        # Extract and print the generated text
        generated_text = response_data.get("response", "")
        # print("Generated Text:", generated_text)
    else:
        print(f"Failed to get a response. Status code: {response.status_code}")
        generated_text = None
    return generated_text

from openai import OpenAI

openai = OpenAI(
    api_key="INSERT_KEY",
    base_url="https://api.deepinfra.com/v1/openai",
)

def generate_question_deep_infra(data, prompt, model = "meta-llama/Meta-Llama-3-8B-Instruct"):
    # print(data)
    chat_completion = openai.chat.completions.create(
    model=model,
    messages=[{"role": "system", "content": "You're generating questions from input data."},
              {"role": "system", "content": prompt},
              {"role": "user", "content": str(data)}],
    stream=False,)

    return chat_completion.choices[0].message.content

def generate_complex_question_deep_infra(data, prompt):
    # print(data)
    chat_completion = openai.chat.completions.create(
    model='meta-llama/Llama-3.3-70B-Instruct',
    max_tokens=None,
    messages=[{"role": "system", "content": "You're generating questions from input data."},
              {"role": "system", "content": prompt},
              {"role": "user", "content": str(data)}],
    stream=False,)

    return chat_completion.choices[0].message.content

def upd_brackets(s):
    """if there're less } than {, add } at the end"""
    if s.count('{')>s.count('}'):
        s+='}'
    return s


clip_questions = {}
CNT=0
for clip_uid, data in filtered_data.items():
    print(f"Generating questions for clip {clip_uid}")
    clip_questions[clip_uid] = {}
    try:
        action_seq = [v for k,v in data.items()]
        purpose_q = generate_complex_question_deep_infra(str(action_seq), PROMPT_PURPOSE)
        purpose_q = json.loads(upd_brackets(purpose_q))
        clip_questions[clip_uid][-1] = {}
        clip_questions[clip_uid][-1]['purpose'] = purpose_q
        CNT+=len(purpose_q)
        print('Successfully generated purpose questions:', len(purpose_q))
    except:
        print('Failed to generate purpose questions')

    for idx, action_data in tqdm(data.items()):
        try:
            _ = action_scores[clip_uid][str(idx)] < 0.5 and 'direct' in action_data
        except:
            continue
        if action_scores[clip_uid][str(idx)] < 0.5 and 'indirect' in action_data:
            clip_questions[clip_uid][idx] = {}
            try:
                # DIRECT QUESTIONS
                action_input = {'verb': action_data['verb'], 'indirect': action_data['indirect']}
                direct_q = generate_question_deep_infra(action_input, PROMPT_DIRECT_QUESTION)
                clip_questions[clip_uid][idx]['direct'] = {'question':direct_q, 'answer': action_data['object']}
                CNT+=1
            except:
                pass

            # INDIRECT QUESTIONS
            indirect_q = []
            input_for_indirect = {'verb': action_data['verb'], 'object': action_data['object']}
            for rel in action_data.get('indirect', '').split(','):
                try:
                    input_for_indirect['answer'] = rel
                    other_rel = action_data.get('indirect', '').replace(rel, '').replace(',', '').strip()
                    if len(other_rel)>2: input_for_indirect['other_relations'] = other_rel
                    ind_q = generate_question_deep_infra(input_for_indirect,
                                                         PROMPT_INDIRECT_QUESTION,
                                                         model='meta-llama/Meta-Llama-3.1-70B-Instruct')
                    indirect_q.append({'question': ind_q, 'answer': rel})
                    CNT+=1
                except:
                    pass
            clip_questions[clip_uid][idx]['indirect'] = indirect_q

            #BEFORE-AFTER QUESTIONS
            try:
                if idx>1:
                    before_q = generate_question_deep_infra(action_data, PROMPT_BEFORE)
                    clip_questions[clip_uid][idx]['before'] = {'question':before_q, 'answer': processed_data[clip_uid][idx-1]['verb']+ ' ' + processed_data[clip_uid][idx-1]['object']}
                    CNT+=1
                after_q = generate_question_deep_infra(action_data, PROMPT_AFTER)
                clip_questions[clip_uid][idx]['after'] = {'question':after_q, 'answer': processed_data[clip_uid][idx+1]['verb']+ ' ' + processed_data[clip_uid][idx+1]['object']}
                CNT+=1
            except:
                pass
    print('Total questions generated:', CNT)

# save clip_questions to json
with open("clip_questions_all_unict.json", "w") as f:
    json.dump(clip_questions, f, indent=2)

Generating questions for clip 49d65bed-9479-427e-bb68-0c9a18560ac6
Successfully generated purpose questions: 3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 100013.21it/s]

Total questions generated: 3
Generating questions for clip 2ae97f2a-b16d-467b-b73a-721cb00996f1


Successfully generated purpose questions: 5


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:05<00:00,  6.10it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 6955.73it/s]

Total questions generated: 1819
Generating questions for clip 9267f964-d5a1-4062-872f-657e2a4cbe81


Successfully generated purpose questions: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8439.24it/s]

Total questions generated: 1820


In [24]:
import re
#read EASG_unict_master_final.json
with open("EASG_unict_master_final.json", "r") as f:
    original_data = json.load(f)

def clean_questions(question):
    """
    Sometimes LLM produce messy results such as "Here is your question: ..."
    this function extracts only question from provided string
    """
    #remove all text after ?
    question = question.split('?')[0]

    question = question[question.find("W"):]
    return question+'?'

def highlight_grounded(question, keywords):
    """
    function takes question and keywords and highlights keywords in the question
    if it's already in < >, do not highlight it
    """
    for keyword in keywords:
        pattern = re.compile(r'(?<!<)(' + re.escape(keyword) + r')(?!>)')
        question = pattern.sub(r'<\1>', question).replace(keyword, f"<{keyword}>")
    return question

def integrate_qa_with_groundings(clip_questions, original_data):
    """
    for each clip function takes clip_questions, and each answer is integrated with grounding from original_data
    original_data[clip_uid]['graphs'] contains the sequence of graphs, where each graph contains 'groundings'.
    For groundings we take 'pnr' frame and 'bbox' for the object which could be found in "answer".
    If there's no such object - we look into 'pre' and 'post' frames.
    If there's no such object in 'pre' and 'post' frames - we leave the gorundings empty

    In the output we should switch from action idx to pnr number from original data.
    """
    integrated_data = {}
    for clip_uid, questions in clip_questions.items():
        integrated_data[clip_uid] = {}
        integrated_data[clip_uid]['purpose'] = questions["-1"]['purpose']
        for idx, question_data in questions.items():
            if int(idx)<0: continue
            pnr = original_data[clip_uid]['graphs'][int(idx)-1]['pnr']
            #for each question-answer pair we need to find the object in the grounding
            # print(pnr, question_data)
            integrated_data[clip_uid][pnr] = {'groundings':{}, 'questions':[]}
            grounded_objects = []
            for q_type, qas in question_data.items():
                #check is qa is a list
                if not isinstance(qas, list):
                    qas = [qas]
                for qa_pair in qas:

                    cleaned_question = clean_questions(qa_pair['question'])
                    #check if any object from original_data[clip_uid]['graphs'][int(idx)-1]['pnr']['bbox'] contains qa_pair['answer']
                    for obj in original_data[clip_uid]['graphs'][int(idx)-1]['groundings']['pnr'].keys():
                        if obj in qa_pair['answer']:
                            integrated_data[clip_uid][pnr]['groundings'][obj] = original_data[clip_uid]['graphs'][int(idx)-1]['groundings']['pnr'][obj]

                        if obj in qa_pair['question']:
                            grounded_objects.append(obj)
                            integrated_data[clip_uid][pnr]['groundings'][obj] = original_data[clip_uid]['graphs'][int(idx)-1]['groundings']['pnr'][obj]
                    
                    integrated_data[clip_uid][pnr]['questions'].append({
                                "type": q_type,
                                "question": highlight_grounded(cleaned_question,grounded_objects),
                                "answer": qa_pair['answer'],
                            })
                        
    return integrated_data

#read clip_questions.json
with open("clip_questions_all_unict.json", "r") as f:
    clip_questions = json.load(f)

integrated_data = integrate_qa_with_groundings(clip_questions, original_data)
print(integrated_data['49d65bed-9479-427e-bb68-0c9a18560ac6'])

#save integrated_data to json
with open("clip_questions_all_unict_grounded.json", "w") as f:
    json.dump(integrated_data, f, indent=2)

{'purpose': {'cardboard': {'question': 'What is the purpose of the cardboard in the video?', 'answer': 'The cardboard is being cut, arranged, cleaned off, and put away, indicating it is being used as a material for a project or task.'}, 'cutter': {'question': 'What is the purpose of the cutter in the video?', 'answer': "The cutter is used to cut the cardboard, as indicated by the actions 'cut cardboard with cutter' and 'cut cardboard in cutter'."}, 'ruler': {'question': 'What is the purpose of the ruler in the video?', 'answer': "The ruler is used to adjust and measure the cardboard, as indicated by the actions 'adjust ruler' and 'cut cardboard on table with ruler'."}}}


In [ ]:
import json

# Load the JSON data (assuming it's stored in a file called "data.json")
with open("EASG_unict_master_final.json", "r") as file:
    data = json.load(file)

# Create the mapping
mapping = {}

for clip_uid, clip_data in data.items():
    if "graphs" in clip_data:
        for graph in clip_data["graphs"]:
            key = (clip_uid, graph["pnr"])
            mapping[key] = graph["graph_uid"]

print(mapping)


In [29]:
import json

# Load integrated data
with open("clip_questions_all_unict_grounded.json", "r") as f:
    integrated_data = json.load(f)

# Load narration mappings
with open("/home/ivan/EASG/input-preparation-v2/narration_annotation_mappings_unict_all.json", "r") as f:
    narrations_raw = json.load(f)

# Build a dictionary from narrations.json: graph_uid -> narration_text
narration_dict = {}
for item in narrations_raw.get("mappings", []):
    for graph_uid, narration_info in item.items():
        narration_dict[graph_uid] = narration_info.get("narration_text", "")

# Update integrated_data with graph_uid and narration_text for each frame key
for clip_uid, clip_data in integrated_data.items():
    for key in list(clip_data.keys()):
        # We expect keys that are frame numbers to be digits. Skip non-frame keys (like "purpose").
        if key.isdigit():
            frame_number = int(key)
            map_key = (clip_uid, frame_number)
            # Check if the mapping exists for this clip and frame
            if map_key in mapping:
                graph_uid = mapping[map_key]
                # Add the graph_uid and narration_text into the frame's dict
                clip_data[key]["graph_uid"] = graph_uid
                clip_data[key]["narration_text"] = narration_dict.get(graph_uid, "")
            else:
                print(f"No mapping found for clip {clip_uid} frame {frame_number}")

# Optionally, write out the updated integrated_data
with open("clip_questions_all_unict_grounded_updated.json", "w") as f:
    json.dump(integrated_data, f, indent=2)

print("Updated integrated_data.json with graph_uid and narration_text.")


No mapping found for clip 2ae97f2a-b16d-467b-b73a-721cb00996f1 frame 20381
No mapping found for clip 2ae97f2a-b16d-467b-b73a-721cb00996f1 frame 21124
No mapping found for clip 2ae97f2a-b16d-467b-b73a-721cb00996f1 frame 24119
No mapping found for clip fc162164-d4c6-4fdb-a511-f3a500ec031e frame 1710
No mapping found for clip fc162164-d4c6-4fdb-a511-f3a500ec031e frame 2452
No mapping found for clip fc162164-d4c6-4fdb-a511-f3a500ec031e frame 3666
No mapping found for clip fc162164-d4c6-4fdb-a511-f3a500ec031e frame 6351
No mapping found for clip fc162164-d4c6-4fdb-a511-f3a500ec031e frame 6885
No mapping found for clip fc162164-d4c6-4fdb-a511-f3a500ec031e frame 7324
No mapping found for clip fc162164-d4c6-4fdb-a511-f3a500ec031e frame 8416
No mapping found for clip 4f68183f-610a-44de-b102-e7f300b49dcd frame 1845
No mapping found for clip 4f68183f-610a-44de-b102-e7f300b49dcd frame 3054
No mapping found for clip 4f68183f-610a-44de-b102-e7f300b49dcd frame 3586
No mapping found for clip 4f68183f-

# Adding attributes to objects in the dataset

In [ ]:
import json
import os
from collections import defaultdict

# Path configuration
MASTER_JSON = "/home/ivan/VQA/EASG_unict_master_final.json"
ANNOTATIONS_DIR = "/home/ivan/EASG/input-preparation-v2/annotations_all/"
OUTPUT_MAPPING = "/home/ivan/frame_mapping.json"

# Load master JSON
with open(MASTER_JSON) as f:
    master_data = json.load(f)

mapping = defaultdict(dict)

for clip_uid, clip_data in master_data.items():
    # Extract graphs for this clip
    for graph in clip_data.get("graphs", []):
        video_frame_number = graph["pnr"]
        graph_uid = graph["graph_uid"]
        
        # Load annotation JSON
        annotation_path = f"{ANNOTATIONS_DIR}/{graph_uid}/{clip_uid}.json"
        if os.path.exists(annotation_path):
            with open(annotation_path) as ann_f:
                annotation = json.load(ann_f)
                print(annotation)
                clip_frame_number = annotation['pnr_frame']['clip_frame_number']
        else:
            clip_frame_number = None
        
        # Add to mapping
        mapping[clip_uid][video_frame_number] = {
            "graph_uid": graph_uid,
            "clip_frame_number": clip_frame_number,
        }

# Save mapping
with open(OUTPUT_MAPPING, 'w') as f:
    json.dump(mapping, f, indent=2)

In [ ]:
import json
from openai import OpenAI

# Initialize the DeepInfra OpenAI client
openai = OpenAI(
    api_key="INSERT_KEY", 
    base_url="https://api.deepinfra.com/v1/openai",
)

def generate_attributes(data, prompt):
    """
    Calls the DeepInfra API using the Llama model to generate attributes from the input data.
    """
    chat_completion = openai.chat.completions.create(
        model='meta-llama/Llama-3.3-70B-Instruct',
        max_tokens=None,
        messages=[
            {"role": "system", "content": "You're generating attributes for objects based on provided data, you only generate json files."},
            {"role": "system", "content": prompt},
            {"role": "user", "content": json.dumps(data)}
        ],
        stream=False,
    )
    return chat_completion.choices[0].message.content


prompt = (
    "Given the following data containing groundings, questions, and narration_text, "
    "identify attributes for the objects present in the groundings that are mentioned in the narration_text. "
    "Return a valid JSON object in the following format: {\"object_name\": \"attribute\"}. "
    "By attribute we imply adjectives or descriptive properties that describe the object. "
    "Note that the attribute can't refer to another object. E.g. if the spoon is on the table, then 'on the table' is NOT an attribute of spoon"
    "The CameraWearer cleans wooden table with cloth in right hand. ---> {\"table\": \"wooden\"}."
    "If no attributes can be extracted, return null. If there are multiple objects, but only few of them has attributes, return only those with attributes."
    "NO ADDITIONAL TEXT SHOULD BE INCLUDED. ONLY JSON or None"
)


with open("clip_questions_all_unict_grounded_updated.json", "r") as f:
    integrated_data = json.load(f)

from tqdm import tqdm

# Process each clip and frame
for clip_uid, clip_data in tqdm(integrated_data.items()):
    # Loop over keys (assuming non-digit keys like "purpose" remain unchanged)
    for key in list(clip_data.keys()):
        if key.isdigit():
            frame_data = clip_data[key]
            # Build data payload for the Llama API
            data_for_llama = {
                "groundings": frame_data.get("groundings", {}),
                "questions": frame_data.get("questions", []),
                "narration_text": frame_data.get("narration_text", "")
            }
            try:
                # Call the API to extract attributes
                response = generate_attributes(data_for_llama, prompt)
                # print(response)
                # Parse the API response into a JSON object
                attributes = json.loads(response)
            except Exception as e:
                print(f"Error processing clip {clip_uid} frame {key}: {e}")
                attributes = None
            # Update the frame data with the attributes (if attributes extraction is not possible, set to None)
            frame_data["attributes"] = attributes

# Write out the updated integrated data with attributes to a new file
with open("integrated_data_updated_with_attributes.json", "w") as f:
    json.dump(integrated_data, f, indent=2)

print("Integrated data has been updated with attributes.")


# Normalize groundings

In [31]:
import json

# Load both JSON files
with open("integrated_data_updated_with_attributes.json", "r") as f:
    data = json.load(f)

with open("EASG_unict_master_final.json", "r") as f:
    dimensions = json.load(f)  # Format: {clip_uid: {"W": ..., "H": ...}}

# Normalize coordinates with cropping of negatives to 0
for clip_uid, content in data.items():
    if clip_uid not in dimensions:
        continue

    W = dimensions[clip_uid]["W"]
    H = dimensions[clip_uid]["H"]

    for key, value in content.items():
        if not key.isdigit():
            continue  # skip non-timestamp keys like "purpose"
        if "groundings" not in value:
            continue

        for obj, bbox in value["groundings"].items():
            if all(k in bbox for k in ("left", "top", "width", "height")):
                bbox["left"] = max(0, bbox["left"]) / W
                bbox["top"] = max(0, bbox["top"]) / H
                bbox["width"] = bbox["width"] / W
                bbox["height"] = bbox["height"] / H

# Save the normalized data
with open("data_updated_with_attributes_normalized.json", "w") as f:
    json.dump(data, f, indent=2)
